# Understanding Loop Phase I Demo

This notebook demonstrates a single pass of the AlphaTrade understanding loop using the Phase I components. It ingests recorded EURUSD market data, fuses the sensory dimensions, produces a belief state and regime signal, applies fast-weight gating around the policy router, and records the resulting decision in the governance diary.


## Setup
The demo loads a small slice of real EURUSD hourly candles captured on 2024-03-04 and 2024-03-05 (see `docs/examples/data/eurusd_hourly_20240304_20240306.csv`). All steps run locally—no external services are required.

Run the cells sequentially from top to bottom.


In [ ]:
from __future__ import annotations

import asyncio
from datetime import datetime, timezone
from pathlib import Path
from pprint import pprint

import pandas as pd

from src.core.event_bus import EventBus
from src.sensory.real_sensory_organ import RealSensoryOrgan
from src.understanding.belief import BeliefBuffer, BeliefEmitter, RegimeFSM
from src.understanding.router import (
    BeliefSnapshot,
    FastWeightAdapter,
    FeatureGate,
    HebbianConfig,
    UnderstandingRouter,
)
from src.thinking.adaptation.policy_router import PolicyRouter, PolicyTactic
from src.understanding.decision_diary import DecisionDiaryStore


In [ ]:
data_path = Path('docs/examples/data/eurusd_hourly_20240304_20240306.csv')
df = pd.read_csv(data_path)
df['timestamp'] = pd.to_datetime(df['timestamp'], utc=True)
df['symbol'] = 'EURUSD'
df.tail()


## Run one understanding loop iteration
The helper coroutine below wires the sensory organs, belief buffer, regime classifier, and understanding router together. It also records the resulting decision in the decision diary under `docs/examples/output/decision_diary_demo.json`.


In [ ]:
async def run_understanding_iteration(frame: pd.DataFrame):
    bus = EventBus()
    await bus.start()
    try:
        organ = RealSensoryOrgan(event_bus=bus)
        snapshot = organ.observe(frame, symbol='EURUSD')

        buffer = BeliefBuffer(belief_id='demo-belief')
        emitter = BeliefEmitter(buffer=buffer, event_bus=bus)
        belief_state = emitter.emit(snapshot)

        regime_fsm = RegimeFSM(event_bus=bus, signal_id='demo-regime')
        regime_signal = regime_fsm.classify(belief_state)
        features = dict(regime_signal.features)

        policy_router = PolicyRouter()
        policy_router.register_tactic(
            PolicyTactic(
                tactic_id='mean_revert',
                base_weight=0.9,
                parameters={'symbol': 'EURUSD', 'side': 'buy', 'size': 100000},
                regime_bias={'bearish': 1.2, 'bullish': 0.7},
                description='Fade moves in balanced regimes',
            )
        )
        policy_router.register_tactic(
            PolicyTactic(
                tactic_id='trend_follow',
                base_weight=0.7,
                parameters={'symbol': 'EURUSD', 'side': 'sell', 'size': 100000},
                regime_bias={'bullish': 1.4},
                description='Ride breakouts when confidence climbs',
            )
        )

        understanding_router = UnderstandingRouter(policy_router)
        understanding_router.register_adapter(
            FastWeightAdapter(
                adapter_id='boost-trend-how',
                tactic_id='trend_follow',
                rationale='Boost breakout tactic when HOW liquidity signal is high',
                feature_gates=(FeatureGate(feature='HOW_signal', minimum=0.03),),
                hebbian=HebbianConfig(
                    feature='HOW_signal',
                    learning_rate=10.0,
                    decay=0.05,
                    floor=1.0,
                    baseline=1.0,
                    ceiling=3.0,
                ),
            )
        )

        belief_snapshot = BeliefSnapshot(
            belief_id=belief_state.belief_id,
            regime_state=regime_signal.regime_state,
            features=features,
            metadata={'symbol': belief_state.symbol},
        )
        decision_bundle = understanding_router.route(belief_snapshot)

        output_dir = Path('docs/examples/output')
        output_dir.mkdir(parents=True, exist_ok=True)
        diary_path = output_dir / 'decision_diary_demo.json'
        if diary_path.exists():
            diary_path.unlink()
        diary_store = DecisionDiaryStore(diary_path, publish_on_record=False)
        diary_entry = diary_store.record(
            policy_id=decision_bundle.decision.tactic_id,
            decision=decision_bundle.decision,
            regime_state=belief_snapshot.regime_state,
            belief_state=belief_state,
            outcomes={'pnl': 0.0, 'confidence': decision_bundle.decision.selected_weight},
            metadata={'demo_run': True},
            notes=('Notebook demo iteration',),
        )
        return {
            'belief_state': belief_state,
            'regime_signal': regime_signal,
            'decision_bundle': decision_bundle,
            'diary_entry': diary_entry,
            'diary_path': diary_path,
        }
    finally:
        await bus.stop()


In [ ]:
results = asyncio.run(run_understanding_iteration(df))
decision = results['decision_bundle'].decision
print('Selected tactic:', decision.tactic_id)
print('Decision weight:', decision.selected_weight)
print('Applied adapters:', results['decision_bundle'].applied_adapters)
print('Fast weight summary:')
pprint(results['decision_bundle'].fast_weight_summary)
print('
Diary written to:', results['diary_path'])


In [ ]:
print(results['diary_entry'].to_markdown())
